In [1]:
import json

In [3]:
with open('../dumping-instagram-6-july-2019.json') as fopen:
    instagram = json.load(fopen)
    
len(instagram)

695571

In [4]:
with open('../dumping-twitter-6-july-2019.json') as fopen:
    twitter = json.load(fopen)
    
len(twitter)

6597867

In [5]:
import cleaning

In [6]:
twitter = twitter + instagram

In [9]:
import re
from tqdm import tqdm

def preprocessing(string):
    string = re.sub(
        'http\S+|www.\S+',
        '',
        ' '.join(
            [i for i in string.split() if i.find('#') < 0 and i.find('@') < 0]
        ),
    )
    
    chars = ',.()!:\'"/;=-'
    for c in chars:
        string = string.replace(c, f' {c} ')
        
    string = re.sub(
        u'[0-9!@#$%^&*()_\-+{}|\~`\'";:?/.>,<]',
        ' ',
        string,
        flags = re.UNICODE,
    )
    string = re.sub(r'[ ]+', ' ', string).strip()
    
    return string.lower()

def loop(strings):
    for i in tqdm(range(len(strings))):
        strings[i] = preprocessing(strings[i])
    return strings

In [10]:
%%time

twitter = cleaning.multiprocessing(twitter, loop)

100%|██████████| 455839/455839 [00:32<00:00, 14181.86it/s]


CPU times: user 4.66 s, sys: 4.27 s, total: 8.92 s
Wall time: 39.3 s


In [11]:
%%time

temp_vocab = list(set(cleaning.multiprocessing(twitter, cleaning.unique_words)))

CPU times: user 3.2 s, sys: 2.17 s, total: 5.37 s
Wall time: 10.8 s


In [12]:
%%time
temp_dict = cleaning.multiprocessing(temp_vocab, cleaning.duplicate_dots_marks_exclamations, list_mode = False)
print(len(temp_dict))

0
CPU times: user 343 ms, sys: 874 ms, total: 1.22 s
Wall time: 2.45 s


In [13]:
%%time

twitter = cleaning.multiprocessing_multiple(twitter, temp_dict, cleaning.string_dict_cleaning)

100%|██████████| 455839/455839 [00:07<00:00, 58558.69it/s]


CPU times: user 4.11 s, sys: 3.58 s, total: 7.68 s
Wall time: 14.7 s


In [14]:
%%time
temp_dict = cleaning.multiprocessing(temp_vocab, cleaning.remove_underscore, list_mode = False)
print(len(temp_dict))

0
CPU times: user 272 ms, sys: 941 ms, total: 1.21 s
Wall time: 1.58 s


In [15]:
%%time

twitter = cleaning.multiprocessing_multiple(twitter, temp_dict, cleaning.string_dict_cleaning)

100%|██████████| 455839/455839 [00:07<00:00, 59117.90it/s]


CPU times: user 4.38 s, sys: 3.66 s, total: 8.04 s
Wall time: 15.5 s


In [16]:
%%time
temp_dict = cleaning.multiprocessing(temp_vocab, cleaning.isolate_spamchars, list_mode = False)
print(len(temp_dict))

12
CPU times: user 271 ms, sys: 999 ms, total: 1.27 s
Wall time: 1.64 s


In [17]:
%%time

twitter = cleaning.multiprocessing_multiple(twitter, temp_dict, cleaning.string_dict_cleaning)

100%|██████████| 455839/455839 [00:07<00:00, 58342.16it/s]


CPU times: user 4.08 s, sys: 3.35 s, total: 7.42 s
Wall time: 14.4 s


In [18]:
%%time
temp_dict = cleaning.multiprocessing(temp_vocab, cleaning.break_short_words, list_mode = False)
print(len(temp_dict))

0
CPU times: user 293 ms, sys: 1.01 s, total: 1.3 s
Wall time: 1.49 s


In [19]:
%%time

twitter = cleaning.multiprocessing_multiple(twitter, temp_dict, cleaning.string_dict_cleaning)

100%|██████████| 455839/455839 [00:07<00:00, 59905.23it/s]


CPU times: user 3.88 s, sys: 3.46 s, total: 7.34 s
Wall time: 14.5 s


In [20]:
%%time
temp_dict = cleaning.multiprocessing(temp_vocab, cleaning.break_long_words, list_mode = False)
print(len(temp_dict))

0
CPU times: user 262 ms, sys: 1.05 s, total: 1.31 s
Wall time: 1.52 s


In [21]:
%%time

twitter = cleaning.multiprocessing_multiple(twitter, temp_dict, cleaning.string_dict_cleaning)

100%|██████████| 455839/455839 [00:07<00:00, 61398.61it/s]


CPU times: user 4.02 s, sys: 3.33 s, total: 7.36 s
Wall time: 14.4 s


In [22]:
%%time
temp_dict = cleaning.multiprocessing(temp_vocab, cleaning.remove_ending_underscore, list_mode = False)
print(len(temp_dict))

0
CPU times: user 267 ms, sys: 1 s, total: 1.27 s
Wall time: 1.46 s


In [23]:
%%time

twitter = cleaning.multiprocessing_multiple(twitter, temp_dict, cleaning.string_dict_cleaning)

100%|██████████| 455839/455839 [00:07<00:00, 59000.42it/s]


CPU times: user 4.11 s, sys: 3.16 s, total: 7.27 s
Wall time: 14.5 s


In [24]:
%%time
temp_dict = cleaning.multiprocessing(temp_vocab, cleaning.remove_starting_underscore, list_mode = False)
print(len(temp_dict))

0
CPU times: user 483 ms, sys: 1.01 s, total: 1.49 s
Wall time: 1.64 s


In [25]:
%%time

twitter = cleaning.multiprocessing_multiple(twitter, temp_dict, cleaning.string_dict_cleaning)

100%|██████████| 455839/455839 [00:07<00:00, 61220.12it/s]


CPU times: user 4.19 s, sys: 3.48 s, total: 7.67 s
Wall time: 14.8 s


In [26]:
%%time
temp_dict = cleaning.multiprocessing(temp_vocab, cleaning.end_punct, list_mode = False)
print(len(temp_dict))

6472
CPU times: user 316 ms, sys: 969 ms, total: 1.28 s
Wall time: 1.49 s


In [27]:
%%time

twitter = cleaning.multiprocessing_multiple(twitter, temp_dict, cleaning.string_dict_cleaning)

100%|██████████| 455839/455839 [00:07<00:00, 57131.13it/s]


CPU times: user 4.28 s, sys: 3.03 s, total: 7.31 s
Wall time: 14.8 s


In [28]:
%%time
temp_dict = cleaning.multiprocessing(temp_vocab, cleaning.start_punct, list_mode = False)
print(len(temp_dict))

6805
CPU times: user 313 ms, sys: 1.02 s, total: 1.34 s
Wall time: 1.51 s


In [29]:
%%time

twitter = cleaning.multiprocessing_multiple(twitter, temp_dict, cleaning.string_dict_cleaning)

100%|██████████| 455839/455839 [00:08<00:00, 56388.80it/s]


CPU times: user 4.18 s, sys: 3.53 s, total: 7.71 s
Wall time: 15.4 s


In [30]:
%%time
temp_dict = cleaning.multiprocessing(temp_vocab, cleaning.join_dashes, list_mode = False)
print(len(temp_dict))

0
CPU times: user 277 ms, sys: 989 ms, total: 1.27 s
Wall time: 1.57 s


In [31]:
%%time

twitter = cleaning.multiprocessing_multiple(twitter, temp_dict, cleaning.string_dict_cleaning)

100%|██████████| 455839/455839 [00:07<00:00, 61595.30it/s]


CPU times: user 4.03 s, sys: 3.17 s, total: 7.19 s
Wall time: 14.1 s


In [32]:
twitter = [s.lower() for s in twitter]

In [33]:
twitter[:100]

['ternyata kl lg sdih bisa ngasilin makanan enak',
 'abu kampret',
 'bapa saya suka pake oppo saya sukanya nokia kaka saya sukanya samsung yg penting punya hape aja',
 'ngelamar kasih cincin tp kok mukanya songong ya sedih gue liatnya',
 'caption iki nggarai uwong males nikah min ya kali manusia arep punah ngunu neg gak nikah',
 'pertanyaannya sederhana jika kami memang dukung prabowo ngapain selama kampanye kemarin capek dukung jokowi sampa',
 '',
 'memiliki sedikit iman lebih berharga dari pada memiliki segudang emas',
 'untuk mengamankan suara partai ahmad rofiq selaku sekjen partai perindo meminta kepada seluruh caleg dan struktur',
 'dom jakpus sih bebas mau ketemuan or shopee',
 'bisa dapet duit ini kaga punya mobil juga kan kaya gemer gemer ini kaga',
 'on jadahnyaaa in sorry bad english hihuheheho',
 'valentino rossi tidak setuju kompetisi motogp dimulai dari eropa',
 'sis tak faham apa yang mungkin ini puncanya tu',
 'martabak terang bulan martabak untuk yg asin gurih a k a ma

In [34]:
with open('ms-socialmedia.json', 'w') as fopen:
    json.dump(twitter, fopen)